In [67]:
from decimal import Decimal
from math import sqrt
from typing import Any


def calcAvg(x):
    sum = Decimal(0)
    for i in x:
        sum = (sum+i).quantize(Decimal(i))
    return (sum/Decimal(len(x))).quantize(Decimal(sum))

def calcSigma(x):
    avg = calcAvg(x)
    sum  = Decimal(0)
    for i in x:
        sum = (sum+(avg-i)**2)
    return (sum/Decimal(len(x)-1).sqrt()).quantize(Decimal(avg))

def calcUncertainty(x, d, c):
    sigma = calcSigma(x)
    delta_a = sigma*c
    delta_b = d/c
    return (delta_a**2 + delta_b**2).sqrt().quantize(Decimal(x[0]))

class ExpData:
    def __init__(self, avg:Decimal, uncertainty:Decimal):
        if not isinstance(avg, Decimal):
            avg = Decimal(avg)
        if not isinstance(uncertainty, Decimal):
            uncertainty = Decimal(uncertainty)
        self.avg = avg
        self.u = uncertainty
    
    def __str__(self):
        return f"avg: {self.avg}, u: {self.u}, ur: {self.u/self.avg}"

    def __repr__(self):
        return f"avg: {self.avg}, u: {self.u}, ur: {self.u/self.avg}"
    
    def __add__(self, other):
        avg = (self.avg + other.avg).quantize(Decimal(self.avg))
        u = (self.u**2 + other.u**2).sqrt().quantize(Decimal(avg))
        return ExpData(avg, u)

    def __sub__(self, other):
        avg = (self.avg - other.avg).quantize(Decimal(self.avg))
        u = (self.u**2 + other.u**2).sqrt().quantize(Decimal(avg))
        return ExpData(avg, u)

    def __mul__(self, other):
        avg = (self.avg*other.avg).quantize(Decimal(self.avg))
        ur1 = self.u/self.avg
        ur2 = other.u/other.avg
        ur = (ur1**2 + ur2**2).sqrt()
        u = (ur*avg).quantize(Decimal(avg))
        return ExpData(avg, u)
    
    def __truediv__(self, other):
        avg = (self.avg/other.avg).quantize(Decimal(self.avg))
        ur1 = self.u/self.avg
        ur2 = other.u/other.avg
        ur = (ur1**2 + ur2**2).sqrt()
        u = (ur*avg).quantize(Decimal(avg))
        return ExpData(avg, u)
    
    def __pow__(self, other):
        avg = (self.avg**other).quantize(Decimal(self.avg))
        ur1 = self.u/self.avg
        ur = abs(ur1*other)
        u = (ur*avg).quantize(Decimal(avg))
        return ExpData(avg, u)


def genData(x:list, d, c=1.05):
    avg = calcAvg(x)
    u = calcUncertainty(x, d, c)
    return ExpData(avg, u)


## 凸透镜
### 自准法

In [50]:
from decimal import Decimal

x_1l = ['20.95', '20.45', '20.45', '20.50', '20.80', '20.45']
x_1r = ['25.80', '25.25', '25.25', '25.40', '25.65', '25.35']
x_2l = ['20.05', '20.15', '20.05', '19.90', '20.20', '19.85']
x_2r = ['24.90', '25.00', '24.90', '24.80', '25.10', '24.80']

x_1l = [Decimal(x) for x in x_1l]
x_1r = [Decimal(x) for x in x_1r]
x_2l = [Decimal(x) for x in x_2l]
x_2r = [Decimal(x) for x in x_2r]

x_1l_avg = calcAvg(x_1l)
x_1r_avg = calcAvg(x_1r)
x_2l_avg = calcAvg(x_2l)
x_2r_avg = calcAvg(x_2r)

pos = ((Decimal('1.20')+Decimal('6.60'))/2+(Decimal('22.96')/10)).quantize(Decimal('1.20'))

x_avg = ((x_1l_avg+x_1r_avg+x_2l_avg+x_2r_avg)/4).quantize(x_1l_avg)

f = (x_avg-pos).quantize(Decimal(x_avg))
f

Decimal('16.55')

In [41]:
from re import U


delta = Decimal('0.05')
c = Decimal('1.05')
n = 6
u_x1l = calcUncertainty(x_1l, delta, c)
u_x1r = calcUncertainty(x_1r, delta, c)
u_x2l = calcUncertainty(x_2l, delta, c)
u_x2r = calcUncertainty(x_2r, delta, c)
[u_x1l, u_x1r, u_x2l, u_x2r]

[Decimal('0.12'), Decimal('0.12'), Decimal('0.06'), Decimal('0.06')]

In [24]:
u_x = (u_x1l**2 + u_x1r**2 + u_x2l**2 + u_x2r**2).sqrt().quantize(u_x1l)
u_xr = (u_x/x_avg)
(u_x, u_xr)

(Decimal('0.19'), Decimal('0.008351648351648351648351648352'))

### 共轭法

In [43]:
x_1ll = ['27.00', '27.00', '27.50', '27.80', '28.10', '28.50']
x_1rr = ['31.80', '31.80', '32.30', '32.60', '33.00', '33.30']
x_2ll = ['54.05', '54.40', '53.45', '52.70', '53.10', '53.10']
x_2rr = ['58.95', '59.20', '58.30', '57.50', '58.00', '58.00']

x_1ll = [Decimal(x) for x in x_1ll]
x_1rr = [Decimal(x) for x in x_1rr]
x_2ll = [Decimal(x) for x in x_2ll]
x_2rr = [Decimal(x) for x in x_2rr]

x_1l = genData(x_1ll, delta, c)
x_1r = genData(x_1rr, delta, c)
x_2l = genData(x_2ll, delta, c)
x_2r = genData(x_2rr, delta, c)
pos = ExpData(pos, 0)
pos

avg: 6.20, u: 0

In [44]:
pos_t = ((Decimal('78.45')+Decimal('82.80'))/2).quantize(Decimal('78.45'))
pos_t = ExpData(pos_t, 0)
pos_t

avg: 80.62, u: 0

In [49]:
D = pos_t - pos
x_1 = (x_1l+x_1r)/ExpData(Decimal(2), 0)
x_2 = (x_2l+x_2r)/ExpData(Decimal(2), 0)
d = x_2-x_1
f = (D**2-d**2)/(ExpData(Decimal(4), 0)*D)
f

avg: 16.34, u: 0.11, ur: 0.006731946144430844553243574051

## 凹透镜
### 物距像距法

In [65]:
x_1ll = ['59.20', '58.95', '59.30', '58.75', '59.30', '59.10']
x_1rr = ['64.10', '63.85', '64.20', '63.65', '64.20', '64.00']
x_2ll = ['59.70', '59.10', '59.65', '58.90', '59.55', '59.15']
x_2rr = ['64.60', '64.00', '64.55', '63.80', '64.45', '64.15']

x_1ll = [Decimal(x) for x in x_1ll]
x_1rr = [Decimal(x) for x in x_1rr]
x_2ll = [Decimal(x) for x in x_2ll]
x_2rr = [Decimal(x) for x in x_2rr]

x_1l = genData(x_1ll, delta, c)
x_1r = genData(x_1rr, delta, c)
x_2l = genData(x_2ll, delta, c)
x_2r = genData(x_2rr, delta, c)

A_1 = ((Decimal('71.40')+Decimal('76.30'))/2).quantize(Decimal('71.40'))
A_2 = ((Decimal('91.75')+Decimal('96.65'))/2).quantize(Decimal('91.75'))
A_1 = ExpData(A_1, 0)
A_2 = ExpData(A_2, 0)

In [70]:
x_1 = (x_1l + x_1r) / ExpData(2, 0)
x_2 = (x_2l + x_2r) / ExpData(2, 0)
x = (x_1 + x_2) / ExpData(2, 0)
s = A_1 - x
s_ = A_2 - x
f = (s**(-1)+s_**(-1))**(-1)
f, s**(-1), s, x

(avg: 9.09, u: 0.00, ur: 0,
 avg: 0.08, u: 0.00, ur: 0,
 avg: 12.17, u: 0.10, ur: 0.008216926869350862777321281841,
 avg: 61.68, u: 0.10, ur: 0.001621271076523994811932555123)